# Alternative Models - Full GoogLeNet Architecture

### Set up

In [1]:
### Load necessary libraries ###
import glob
import os
import librosa
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout, Input
from keras.models import Model

import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from librosa import display
import librosa

C:\Users\EmmaC\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
x_train = pd.read_csv("allFeaturesData/train_data.csv").to_numpy().reshape(7894, 40, 5, 1)
x_test = pd.read_csv("allFeaturesData/test_data.csv").to_numpy().reshape(836, 40, 5, 1)
y_train = pd.read_csv("allFeaturesData/train_labels.csv").to_numpy().reshape(7894,)
y_test = pd.read_csv("allFeaturesData/test_labels.csv").to_numpy().reshape(836,)

In [3]:
pool_size = (2, 2)
kernel_size = (3, 3)
input_shape = (40, 5, 1)
num_classes = 10

In [4]:
# # model
# model = keras.Sequential()
# model.add(keras.layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(40,5,1),padding='same'))
# model.add(keras.layers.AveragePooling2D(padding='same'))
# model.add(keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu',padding='same'))
# model.add(keras.layers.AveragePooling2D(padding='same'))
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(units=120, activation='relu'))
# model.add(keras.layers.Dense(units=84, activation='relu'))
# model.add(keras.layers.Dense(units=10, activation = 'softmax'))


# optimizer = keras.optimizers.Adam(lr=1e-4)
# model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [5]:
# keras.backend.clear_session()
# model.fit(x_train, y_train, epochs = 50, batch_size = 50, validation_data = (x_test, y_test))

In [6]:
# predictions = model.predict(x_test)
# predictions = np.apply_along_axis(lambda x : list(x).index(max(x)),1, predictions)
# predictions.shape

In [7]:
# pd.DataFrame({
#     "prediction" : predictions,
#     "actual" : y_test
# }).groupby("actual") \
#     .apply(lambda x:x["prediction"]==x["actual"]) \
#     .groupby("actual") \
#     .agg(Accuracy = 'mean') \
#     .reset_index() \
#     .assign(names=[
#         "air_conditioner",
#         "car_horn",
#         "children_playing",
#         "dog_bark",
#         "drilling",
#         "engine_idling",
#         "gun_shot",
#         "jackhammer",
#         "siren",
#         "street_music"
#     ])

### GoogleNet

In [8]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.layers import concatenate
import cv2 
from keras.utils import np_utils

In [9]:
img_rows = 224
img_cols = 224

In [10]:
# Resize training images
x_train = np.array([cv2.resize(img, (img_rows,img_cols),3) for img in x_train[:,:,:,:]])
x_test = np.array([cv2.resize(img, (img_rows,img_cols),3) for img in x_test[:,:,:,:]])
# np_array.reshape((im_height, im_width, 3))

In [11]:
x_train.shape

(7894, 224, 224)

In [12]:
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

In [13]:
x_train.shape

(7894, 224, 224, 1)

In [14]:
# x_train = np.array([cv2.resize(img, (img_rows,img_cols),3) for img in x_train[:,:,:,:]])
z = np.zeros(x_train.shape[:-1] + (2,), dtype=x_train.dtype)
x_train = np.concatenate((x_train, z), axis=-1)

In [15]:
# x_train = np.array([cv2.resize(img, (img_rows,img_cols),3) for img in x_train[:,:,:,:]])
z = np.zeros(x_test.shape[:-1] + (2,), dtype=x_test.dtype)
x_test = np.concatenate((x_test, z), axis=-1)

In [16]:
x_train.shape

(7894, 224, 224, 3)

In [17]:
x_test.shape

(836, 224, 224, 3)

In [18]:
# Transform targets to keras compatible format
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# preprocess data
x_train = x_train / 255.0
x_test = x_test / 255.0

In [19]:
def Inception_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4): 
  # Input: 
  # - f1: number of filters of the 1x1 convolutional layer in the first path
  # - f2_conv1, f2_conv3 are number of filters corresponding to the 1x1 and 3x3 convolutional layers in the second path
  # - f3_conv1, f3_conv5 are the number of filters corresponding to the 1x1 and 5x5  convolutional layer in the third path
  # - f4: number of filters of the 1x1 convolutional layer in the fourth path

  # 1st path:
  path1 = Conv2D(filters=f1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)

  # 2nd path
  path2 = Conv2D(filters = f2_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path2 = Conv2D(filters = f2_conv3, kernel_size = (3,3), padding = 'same', activation = 'relu')(path2)

  # 3rd path
  path3 = Conv2D(filters = f3_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path3 = Conv2D(filters = f3_conv5, kernel_size = (5,5), padding = 'same', activation = 'relu')(path3)

  # 4th path
  path4 = MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
  path4 = Conv2D(filters = f4, kernel_size = (1,1), padding = 'same', activation = 'relu')(path4)

  output_layer = concatenate([path1, path2, path3, path4], axis = -1)

  return output_layer


In [20]:
def GoogLeNet():
  # input layer 
    input_layer = Input(shape = (224, 224, 3))
    # input_layer = Input(shape=(40, 5, 1))

  # convolutional layer: filters = 64, kernel_size = (7,7), strides = 2
    # X = Conv2D(filters = 64, kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu')(input_layer)
    X = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2')(input_layer)

    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

    # convolutional layer: filters = 64, strides = 1
    X = Conv2D(filters = 64, kernel_size = (1,1), strides = 1, padding = 'same', activation = 'relu')(X)

    # convolutional layer: filters = 192, kernel_size = (3,3)
    X = Conv2D(filters = 192, kernel_size = (3,3), padding = 'same', activation = 'relu')(X)

    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

    # 1st Inception block
    X = Inception_block(X, f1 = 64, f2_conv1 = 96, f2_conv3 = 128, f3_conv1 = 16, f3_conv5 = 32, f4 = 32)

    # 2nd Inception block
    X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 192, f3_conv1 = 32, f3_conv5 = 96, f4 = 64)

    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

    # 3rd Inception block
    X = Inception_block(X, f1 = 192, f2_conv1 = 96, f2_conv3 = 208, f3_conv1 = 16, f3_conv5 = 48, f4 = 64)

    # Extra network 1:
    X1 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
    X1 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X1)
    X1 = Flatten()(X1)
    X1 = Dense(1024, activation = 'relu')(X1)
    X1 = Dropout(0.7)(X1)
    X1 = Dense(10, activation = 'softmax')(X1)


    # 4th Inception block
    X = Inception_block(X, f1 = 160, f2_conv1 = 112, f2_conv3 = 224, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

    # 5th Inception block
    X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 256, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

    # 6th Inception block
    X = Inception_block(X, f1 = 112, f2_conv1 = 144, f2_conv3 = 288, f3_conv1 = 32, f3_conv5 = 64, f4 = 64)

    # Extra network 2:
    X2 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
    X2 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X2)
    X2 = Flatten()(X2)
    X2 = Dense(1024, activation = 'relu')(X2)
    X2 = Dropout(0.7)(X2)
    X2 = Dense(10, activation = 'softmax')(X2)


    # 7th Inception block
    X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, 
                      f3_conv5 = 128, f4 = 128)

    # max-pooling layer: pool_size = (3,3), strides = 2
    X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

    # 8th Inception block
    X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, f3_conv5 = 128, f4 = 128)

    # 9th Inception block
    X = Inception_block(X, f1 = 384, f2_conv1 = 192, f2_conv3 = 384, f3_conv1 = 48, f3_conv5 = 128, f4 = 128)

    # Global Average pooling layer 
    X = GlobalAveragePooling2D(name = 'GAPL')(X)

    # Dropoutlayer 
    X = Dropout(0.4)(X)

    # output layer 
    X = Dense(10, activation = 'softmax')(X)

    # model
    model = Model(input_layer, [X, X1, X2], name = 'GoogLeNet')

    return model

In [21]:
model = GoogLeNet()

In [22]:
#model.summary()

In [23]:
optimizer = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [24]:
keras.backend.clear_session()
model.fit(x_train, y_train, epochs = 50, batch_size = 50, validation_data = (x_test, y_test))

Epoch 1/50
158/158 [==============================] - 307s 2s/step - loss: 6.4707 - dense_4_loss: 2.1697 - dense_1_loss: 2.1474 - dense_3_loss: 2.1536 - dense_4_accuracy: 0.1818 - dense_1_accuracy: 0.1880 - dense_3_accuracy: 0.1923 - val_loss: 6.1331 - val_dense_4_loss: 2.0604 - val_dense_1_loss: 2.0382 - val_dense_3_loss: 2.0346 - val_dense_4_accuracy: 0.2093 - val_dense_1_accuracy: 0.2249 - val_dense_3_accuracy: 0.2285
Epoch 2/50
158/158 [==============================] - 300s 2s/step - loss: 5.9797 - dense_4_loss: 2.0144 - dense_1_loss: 1.9739 - dense_3_loss: 1.9914 - dense_4_accuracy: 0.2222 - dense_1_accuracy: 0.2582 - dense_3_accuracy: 0.2420 - val_loss: 5.5687 - val_dense_4_loss: 1.9000 - val_dense_1_loss: 1.8297 - val_dense_3_loss: 1.8390 - val_dense_4_accuracy: 0.2715 - val_dense_1_accuracy: 0.3313 - val_dense_3_accuracy: 0.3206
Epoch 3/50
158/158 [==============================] - 299s 2s/step - loss: 5.5034 - dense_4_loss: 1.8676 - dense_1_loss: 1.7896 - dense_3_loss: 1.8462